In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# ===================================================================
#  Library
# ===================================================================
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter("ignore")

import numpy as np
from scipy.optimize import minimize

In [4]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_seeds = 3
    n_trials = 2000
    save_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/exp/"
    data_dir = "/content/drive/MyDrive/Colab Notebooks/signate2023/"
    filename = "exp00067"

In [5]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [6]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')

In [7]:
first_stage_kun_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052.csv')
first_stage_kun_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00052_oof_pred.csv')
first_stage_yuji_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp050.csv')
first_stage_yuji_oof = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp050.csv')

In [8]:
first_stage_yuji_oof

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,...,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29
0,2,2954.247573,2735.110086,3288.647709,2893.840210,3253.350282,2885.690295,3202.103770,2799.365349,2989.931642,...,3016.016944,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537
1,25,7931.462351,6944.616443,7214.909240,7252.552211,7198.146932,6827.863480,7136.463291,7656.661603,7039.997672,...,9115.080046,7887.280768,8547.342936,9035.875075,7761.508055,8061.390855,8203.344234,8173.732933,7975.553091,8406.858766
2,26,5843.172706,5827.250371,6115.096360,5831.436096,5819.394501,6109.653073,5937.286683,5893.277859,6008.794932,...,5054.213484,6281.925529,6068.193517,6200.966336,6186.794549,5696.564677,6112.343101,5137.275113,6044.849594,5734.620810
3,51,2857.775866,3369.341416,3117.966141,3300.098540,3334.328270,3239.576316,3168.865643,2931.111896,3368.678163,...,3187.546183,3304.315439,3533.114755,2953.857168,3284.750592,2991.029385,3127.115066,3708.522395,3470.317170,3646.763271
4,52,8220.237821,7545.993230,8159.395741,8416.911652,7936.652895,7614.765389,8401.374941,7866.482215,8084.295455,...,8512.519923,8341.592023,8151.374086,8286.896299,8189.679791,8237.443803,8623.343583,8031.293955,8567.124566,8888.287414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27476,6982.456164,6547.126657,6272.924314,6883.816792,6689.595713,6897.962840,7287.918787,6927.718233,6960.224224,...,6356.229524,7675.862802,7350.344636,7381.649600,7214.676789,7505.790714,6249.015218,6607.377546,7018.185493,6836.033306
27528,27496,7991.618468,7182.188171,6341.634653,7276.265285,7334.970227,7505.917690,8018.629653,7993.739532,7144.644201,...,7081.628705,7235.961181,7564.032880,7547.732790,7510.563210,7233.026233,8282.783772,6893.004144,6148.318192,7514.557007
27529,27506,5841.883459,6062.253497,6325.240063,6711.844782,5872.272990,5930.771389,5433.789309,5979.625795,5746.513645,...,6278.319914,5573.420082,5456.029238,5834.957974,5159.326299,5072.285643,7017.594320,7085.565618,5748.661191,6125.131678
27530,27516,7885.338899,6749.518405,7498.278543,8136.739409,8110.431959,7111.222431,7208.258953,7391.535836,7542.271496,...,6937.205673,7397.558048,9834.556421,7935.070749,8279.184422,8756.414925,9658.642707,6616.694161,8576.559219,7599.898657


In [9]:
kun_exp00055_oof_pred= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00055_oof_pred.csv').rename(columns={'optimized_pred': 'kun_pred_10'})
kun_exp00056_oof_pred = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00056_oof_pred.csv').rename(columns={'oof_pred': 'kun_pred_11'})
second_stage_kun_oof =  kun_exp00055_oof_pred.merge(kun_exp00056_oof_pred, on='id')

oof_df_exp051 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp051.csv').rename(columns={'pred': 'pred_30'})
oof_df_exp052 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp052.csv').rename(columns={'pred': 'pred_31'})
oof_df_exp053 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp053.csv').rename(columns={'pred': 'pred_32'})
oof_df_exp054 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/oof_df_exp054.csv').rename(columns={'pred': 'pred_33'})
second_stage_yuji_oof = pd.concat([oof_df_exp051, oof_df_exp052['pred_31'], oof_df_exp053['pred_32'], oof_df_exp054['pred_33']],axis=1)


kun_exp00055 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00055.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_10'})
kun_exp00056 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/kun_exp00056.csv', header=None).rename(columns={0:'id', 1: 'kun_pred_11'})
second_stage_kun_test =  kun_exp00055.merge(kun_exp00056, on='id')

exp051 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp051.csv', header=None).rename(columns={0:'id', 1: 'pred_30'})
exp052 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp052.csv', header=None).rename(columns={0:'id', 1: 'pred_31'})
exp053 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp053.csv', header=None).rename(columns={0:'id', 1: 'pred_32'})
exp054 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp/exp054.csv', header=None).rename(columns={0:'id', 1: 'pred_33'})
second_stage_yuji_test = pd.concat([exp051, exp052['pred_31'], exp053['pred_32'], exp054['pred_33']],axis=1)

In [10]:
first_stage_yuji_oof

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,...,pred_20,pred_21,pred_22,pred_23,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29
0,2,2954.247573,2735.110086,3288.647709,2893.840210,3253.350282,2885.690295,3202.103770,2799.365349,2989.931642,...,3016.016944,3167.730032,2929.953707,2957.126627,3618.763534,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537
1,25,7931.462351,6944.616443,7214.909240,7252.552211,7198.146932,6827.863480,7136.463291,7656.661603,7039.997672,...,9115.080046,7887.280768,8547.342936,9035.875075,7761.508055,8061.390855,8203.344234,8173.732933,7975.553091,8406.858766
2,26,5843.172706,5827.250371,6115.096360,5831.436096,5819.394501,6109.653073,5937.286683,5893.277859,6008.794932,...,5054.213484,6281.925529,6068.193517,6200.966336,6186.794549,5696.564677,6112.343101,5137.275113,6044.849594,5734.620810
3,51,2857.775866,3369.341416,3117.966141,3300.098540,3334.328270,3239.576316,3168.865643,2931.111896,3368.678163,...,3187.546183,3304.315439,3533.114755,2953.857168,3284.750592,2991.029385,3127.115066,3708.522395,3470.317170,3646.763271
4,52,8220.237821,7545.993230,8159.395741,8416.911652,7936.652895,7614.765389,8401.374941,7866.482215,8084.295455,...,8512.519923,8341.592023,8151.374086,8286.896299,8189.679791,8237.443803,8623.343583,8031.293955,8567.124566,8888.287414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27476,6982.456164,6547.126657,6272.924314,6883.816792,6689.595713,6897.962840,7287.918787,6927.718233,6960.224224,...,6356.229524,7675.862802,7350.344636,7381.649600,7214.676789,7505.790714,6249.015218,6607.377546,7018.185493,6836.033306
27528,27496,7991.618468,7182.188171,6341.634653,7276.265285,7334.970227,7505.917690,8018.629653,7993.739532,7144.644201,...,7081.628705,7235.961181,7564.032880,7547.732790,7510.563210,7233.026233,8282.783772,6893.004144,6148.318192,7514.557007
27529,27506,5841.883459,6062.253497,6325.240063,6711.844782,5872.272990,5930.771389,5433.789309,5979.625795,5746.513645,...,6278.319914,5573.420082,5456.029238,5834.957974,5159.326299,5072.285643,7017.594320,7085.565618,5748.661191,6125.131678
27530,27516,7885.338899,6749.518405,7498.278543,8136.739409,8110.431959,7111.222431,7208.258953,7391.535836,7542.271496,...,6937.205673,7397.558048,9834.556421,7935.070749,8279.184422,8756.414925,9658.642707,6616.694161,8576.559219,7599.898657


In [11]:
second_stage_yuji_oof

,id,pred_30,pred_31,pred_32,pred_33
0,0,8759.290471,8929.546032,8916.322165,8977.757097
1,1,3859.910547,3825.508199,3822.104706,3876.047282
2,2,3027.760649,3049.493176,2982.516292,3056.648628
3,3,7990.517289,8005.600906,7993.295606,7894.338559
4,4,4524.287997,4479.006617,4569.630683,4570.899182
...,...,...,...,...,...
27527,27527,12813.569454,12748.693982,12889.434904,13184.273766
27528,27528,6735.046172,6676.772957,6772.174105,6772.583423
27529,27529,14571.234351,14813.291564,14599.654721,14877.525968
27530,27530,6651.976275,6762.372206,6798.175222,6822.692681


In [12]:
first_stage_kun_oof

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,kun_pred_9
0,0,7118.9443,7627.1987,7084.0015,7043.0950,7122.7530,7335.2310,7350.0060,9836.8750,8781.6880,8747.9750
1,1,3660.8042,3526.6816,3732.6855,3593.2502,3596.9211,3562.2925,3922.8992,3761.7544,3852.5068,3476.0593
2,2,2883.7390,3149.7769,2995.7760,3038.0789,2876.8310,2982.5576,2787.5325,2911.1800,3172.8408,3092.2769
3,3,8650.6270,8943.9030,9243.1750,8973.2610,9158.7180,8505.1610,8713.2310,8287.6740,8649.6400,8874.9210
4,4,4031.1462,4406.5480,4107.9190,4059.3975,4229.2666,4429.8930,4209.0796,4026.7249,4228.1797,4164.2420
...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12879.4610,13338.9980,12310.2180,12790.3090,12681.5170,12661.3610,11953.5220,12138.8550,12707.8850,11684.3060
27528,27528,6693.9270,6920.0894,6770.7925,6634.9660,7131.3125,6796.1294,6812.7780,8508.7300,7977.3506,6693.7810
27529,27529,14174.7040,15265.7090,14390.8955,14423.8730,12980.9630,14560.7450,14018.7240,13443.1000,13441.5670,12847.0780
27530,27530,7086.4136,6367.2173,6483.3950,7116.9200,7356.8930,7240.1910,7432.3013,7450.1953,7293.5034,7242.7275


In [13]:
second_stage_kun_oof

,id,kun_pred_10,kun_pred_11
0,0,8805.219623,7895.4160
1,1,3799.844508,4177.3580
2,2,2959.861854,2818.1372
3,3,7934.654500,8179.8460
4,4,4461.927715,4321.6885
...,...,...,...
27527,27527,12385.902685,12967.0370
27528,27528,6939.817355,6854.7417
27529,27529,14186.293321,15186.2150
27530,27530,6658.054764,6914.8745


In [14]:
kun_oof = first_stage_kun_oof.merge(second_stage_kun_oof, on='id')
kun_test = first_stage_kun_test.merge(second_stage_kun_test, on='id')
yuji_oof = first_stage_yuji_oof.merge(second_stage_yuji_oof, on='id')
yuji_test = first_stage_yuji_test.merge(second_stage_yuji_test, on='id')

In [15]:
yuji_test

,id,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,...,pred_24,pred_25,pred_26,pred_27,pred_28,pred_29,pred_30,pred_31,pred_32,pred_33
0,27532,9665.240676,9060.741324,8931.822504,9993.239922,9387.607212,9908.462741,10128.928844,9738.111509,10579.697128,...,9498.316353,8687.940060,9208.293004,8356.975825,9330.770219,9692.941277,9681.016732,9631.179775,9535.601879,9589.502294
1,27533,5140.878681,4974.611255,4485.057773,5533.795617,5044.527731,4849.379307,5257.737182,5354.979862,5545.143501,...,5628.653785,6202.828111,4785.541393,5604.598269,5398.402382,5744.756995,5589.905122,5625.972274,5579.913722,5696.370941
2,27534,5628.236365,5804.522418,5796.571207,5726.217723,5822.816944,5759.308970,5798.455075,5419.099745,5595.549648,...,5732.867996,5793.773542,5632.035892,5905.933910,5561.372542,5972.185238,5588.496733,5559.707831,5556.921123,5567.494508
3,27535,18624.817802,20021.162475,18835.327938,22007.952770,18960.034958,19900.729462,19313.421537,18475.892150,19846.987451,...,17604.417956,17961.093949,12794.552170,20108.010799,19183.244173,19561.667694,18539.014190,18585.315662,18690.825713,18540.038374
4,27536,3625.637392,4375.456011,4220.231775,3687.370209,4082.474807,4105.429409,3789.946321,3915.428485,4189.230892,...,3830.999498,3871.618438,6658.960126,4595.955768,4849.609894,4540.888135,4461.198006,4460.135571,4474.330698,4508.779351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,14710.830942,11584.438744,11947.372568,13550.423259,13646.679093,13303.559398,14155.207461,15621.013652,13879.446250,...,13841.399729,12156.012675,11113.945897,13684.552604,13453.234406,12058.316829,13135.222052,12961.122998,12965.397426,13145.010785
27533,55065,8852.779860,8068.844513,8619.407623,9115.885754,8196.746981,7947.659290,8410.227937,8646.502454,8689.880353,...,9408.583043,9972.958526,10205.926610,8689.446574,10127.392831,8915.794126,9473.916681,9638.115991,9440.222225,9526.790475
27534,55066,6288.760858,6135.556787,6508.548628,6240.533655,5993.522277,5874.218495,6160.884478,6478.396730,6060.177626,...,5394.629396,5613.617670,6456.335598,5536.471253,5446.296301,5800.008342,5564.120273,5513.388636,5554.669582,5468.090477
27535,55067,5168.341070,5594.025322,5106.247195,4975.679965,5220.941190,5072.216882,4967.672649,5048.859101,5397.634268,...,4919.544215,5329.443448,4729.350399,5490.082143,4927.573842,4721.779342,4795.089266,4829.818804,4832.357339,4831.827553


In [16]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')
df = pd.concat([kun_oof.merge(yuji_oof, on='id'), df_train['price']], axis=1)
test = kun_test.merge(yuji_test, on='id')

In [17]:
df

,id,kun_pred_0,kun_pred_1,kun_pred_2,kun_pred_3,kun_pred_4,kun_pred_5,kun_pred_6,kun_pred_7,kun_pred_8,...,pred_25,pred_26,pred_27,pred_28,pred_29,pred_30,pred_31,pred_32,pred_33,price
0,0,7118.9443,7627.1987,7084.0015,7043.0950,7122.7530,7335.2310,7350.0060,9836.8750,8781.6880,...,8272.276158,5876.657800,6016.322090,7374.013382,8496.804984,8759.290471,8929.546032,8916.322165,8977.757097,27587
1,1,3660.8042,3526.6816,3732.6855,3593.2502,3596.9211,3562.2925,3922.8992,3761.7544,3852.5068,...,3615.842705,4245.317355,4453.437876,4408.398403,3900.820201,3859.910547,3825.508199,3822.104706,3876.047282,4724
2,2,2883.7390,3149.7769,2995.7760,3038.0789,2876.8310,2982.5576,2787.5325,2911.1800,3172.8408,...,3022.052454,3913.586712,3705.560742,2921.102463,3116.785537,3027.760649,3049.493176,2982.516292,3056.648628,10931
3,3,8650.6270,8943.9030,9243.1750,8973.2610,9158.7180,8505.1610,8713.2310,8287.6740,8649.6400,...,8115.355025,8157.420044,7434.588221,8284.727211,7855.624335,7990.517289,8005.600906,7993.295606,7894.338559,16553
4,4,4031.1462,4406.5480,4107.9190,4059.3975,4229.2666,4429.8930,4209.0796,4026.7249,4228.1797,...,4254.960815,4031.139467,3972.143434,4027.789862,4720.410259,4524.287997,4479.006617,4569.630683,4570.899182,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12879.4610,13338.9980,12310.2180,12790.3090,12681.5170,12661.3610,11953.5220,12138.8550,12707.8850,...,11171.113895,10396.341985,14458.313885,14216.093476,12448.334050,12813.569454,12748.693982,12889.434904,13184.273766,32212
27528,27528,6693.9270,6920.0894,6770.7925,6634.9660,7131.3125,6796.1294,6812.7780,8508.7300,7977.3506,...,6987.163770,9398.511437,6896.820540,6121.882704,6568.329712,6735.046172,6676.772957,6772.174105,6772.583423,5400
27529,27529,14174.7040,15265.7090,14390.8955,14423.8730,12980.9630,14560.7450,14018.7240,13443.1000,13441.5670,...,13611.897758,14921.987650,15042.231812,14166.439728,15120.009430,14571.234351,14813.291564,14599.654721,14877.525968,22227
27530,27530,7086.4136,6367.2173,6483.3950,7116.9200,7356.8930,7240.1910,7432.3013,7450.1953,7293.5034,...,6747.818100,7189.621305,7638.963008,6865.697187,6743.532957,6651.976275,6762.372206,6798.175222,6822.692681,3054


In [18]:


# あなたの予測結果の列名のプレフィックスと数
kun_prefix = "kun_pred_"
kun_num = 12

# もう一人の人の予測結果の列名のプレフィックスと数
other_prefix = "pred_"
other_num = 34

def objective(weights):
    kun_preds = [df[f"{kun_prefix}{idx}"] * weights[idx] for idx in range(kun_num)]
    other_preds = [df[f"{other_prefix}{idx}"] * weights[idx+kun_num] for idx in range(other_num)]

    df["pred"] = sum(kun_preds + other_preds)

    score = get_score(y_true=df["price"], y_pred=df["pred"])
    return score

# 初期の重みを設定（すべての予測の重みを等しくする）
initial_weights = [1 / (kun_num + other_num)] * (kun_num + other_num)

result = minimize(objective, initial_weights, method="nelder-mead")

best_weights = result.x

print("Optimized weights:", best_weights)
print("Best score with optimized weights:", result.fun)

Optimized weights: [ 0.05191275  0.00589207  0.16696146  0.08884469 -0.1559621  -0.16574525
 -0.06850776  0.07789791 -0.04325756 -0.21605587  0.13028662  0.6139788
  0.0020418  -0.14680385  0.04144643 -0.08114806 -0.00595006  0.05399517
  0.01782313  0.10493312  0.00895702 -0.09474311  0.07373738  0.05609459
  0.07041251 -0.05099867  0.02205185  0.02901157  0.03892438 -0.10947895
 -0.02139256 -0.03285923 -0.02844089  0.01506704 -0.01243971  0.00489035
  0.04007598 -0.09567638  0.0052151   0.04880734  0.04442813  0.1007086
  0.01947455  0.14135208  0.23055569  0.02057477]
Best score with optimized weights: 43.46088582832526


In [19]:
# 最適化された重みを使用してOOFデータの予測を計算
kun_oof_preds = [df[f"{kun_prefix}{idx}"] * best_weights[idx] for idx in range(kun_num)]
other_oof_preds = [df[f"{other_prefix}{idx}"] * best_weights[idx+kun_num] for idx in range(other_num)]
df["optimized_pred"] = sum(kun_oof_preds + other_oof_preds)

# 結果をCSVファイルとして保存
optimized_oof_filepath = CFG.save_dir + f"kun_{CFG.filename}_oof_pred.csv"
df[["id", "optimized_pred"]].to_csv(optimized_oof_filepath, index=False)

In [20]:
pd.read_csv(optimized_oof_filepath)

,id,optimized_pred
0,0,8590.224386
1,1,4265.354882
2,2,2940.313434
3,3,7883.688680
4,4,4388.798777
...,...,...
27527,27527,13112.154672
27528,27528,6921.545860
27529,27529,15511.980145
27530,27530,6386.621719


In [21]:
get_score(df['price'], pd.read_csv(optimized_oof_filepath)['optimized_pred'])

43.46088582832526

In [22]:
# 最適化された重みを使用してテストデータの予測を計算
kun_preds = [test[f"kun_pred_{idx}"] * best_weights[idx] for idx in range(12)]
other_preds = [test[f"pred_{idx}"] * best_weights[idx+kun_num] for idx in range(34)]
test["pred"] = sum(kun_preds + other_preds)

# 結果をCSVファイルとして保存
test[["id", "pred"]].to_csv(CFG.save_dir + f"kun_{CFG.filename}.csv", index=False, header=None)